<a href="https://colab.research.google.com/github/sajidcsecu/polarImage/blob/main/PolarImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 #linear polar
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import geometric_transform


class PolarImageTransformation:

    #### developed algorithm
    def cartesian_to_polar_image(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2
        polar_image = np.zeros_like(image)
        u = np.arange(width)
        v = np.arange(height)
        # Create grids for radius (r) and angle (θ)
        ru = (u - center_x) * 2 / width
        thetav = (center_y - v) * 2 * np.pi * 2 / height

        r,theta = np.meshgrid(ru,thetav, indexing='xy')
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        # Compute Cartesian coordinates from polar coordinates
        i = (x * width/2 + center_x).astype(int)    # Shape: (height, width)
        j = (y * height/2 + center_y).astype(int) # Shape: (height, width)
        # Ensure coordinates are within image bounds
        i = np.clip(i, 0, width - 1)
        j = np.clip(j, 0, height - 1)
        # Map Cartesian image to polar image using advanced indexing
        polar_image[:, :] = image[j, i]
        return polar_image,r,theta


    def polar_to_cartesian_image(self,image):
        height, width = image.shape
        center_x = width // 2
        center_y = height // 2
        car_image = np.zeros_like(image)
        i = np.arange(width)
        j = np.arange(height)

        x = (i -center_x) * 2 / width
        y= (center_y-j) * 2 / height
        x,y = np.meshgrid(x,y, indexing='xy')
        ru = np.sqrt(x ** 2 + y ** 2)
        #ru[ru >= 1] = 0.
        # print(ru)
        thetav = np.arctan2(y, x, where=True)
        u = (center_x + (ru * width) / 2).astype(int)
        v = (center_y + (thetav * height / (2 * np.pi * 2))).astype(int)

        # Ensure coordinates are within image bounds
        u = np.clip(u, 0, width - 1)
        v = np.clip(v, 0, height - 1)

        # Map Cartesian image to polar image using advanced indexing
        car_image[:, :] = image[v, u]
        return car_image

##### Algorithm on paper Zhang
# Fast Image reconstruction method using radial harmonic fourier moments
    def car_to_pol_zhang(self, image, s=4):
        height, width = image.shape
        swidth = s * width
        sheight = s * height
        dim = (swidth, sheight)
        resized = cv2.resize(image.astype(float), dim, interpolation=cv2.INTER_LINEAR)
        center_x, center_y = swidth // 2, sheight // 2
        polar_image = np.zeros_like(resized)
        # Create grids for radius (r) and angle (θ)
        u = np.arange(sheight)
        ru = u / sheight
        v = np.arange(swidth)
        thetav = 2 * np.pi * v / swidth
        t, r = np.meshgrid(thetav, ru, indexing='xy')
        x = np.ceil(r * swidth * np.cos(t) / 2).astype(int)
        y = np.ceil(r * sheight * np.sin(t) / 2).astype(int)
        p = -x + center_x + 1
        q = y + center_y
        p = np.clip(p, 0, swidth - 1)
        q = np.clip(q, 0, sheight - 1)
        polar_image[:, :] = resized[q, p]
        dim = (width, height)
        polar_image = cv2.resize(polar_image.astype(float), dim, interpolation=cv2.INTER_LINEAR)
        return polar_image

    def pol_to_car_zhang(self, image, s=4):
        height, width = image.shape
        swidth = s * width
        sheight = s * height
        dim = (swidth, sheight)
        resized = cv2.resize(image.astype(float), dim, interpolation=cv2.INTER_LINEAR)
        center_x, center_y = swidth // 2, sheight // 2
        cart_image = np.zeros_like(resized)
        p = (np.arange(swidth) - center_x + 1) * 2 / swidth  # to make perfect square using +1
        q = (center_y - np.arange(sheight)) * 2 / sheight
        p, q = np.meshgrid(p, q, indexing="xy")
        ru = np.sqrt(p ** 2 + q ** 2)
        ru[ru > 1] = 0
        thetav = np.arctan2(q, p, where=True)
        # thetav[thetav < 0] = thetav[thetav < 0] + 2 * np.pi
        ru_index = (ru * sheight / 2).astype(int)
        theta_index = ((thetav + np.pi) * swidth / (2 * np.pi)).astype(int)
        i = np.clip(ru_index, 0, swidth - 1)
        j = np.clip(theta_index, 0, sheight - 1)
        cart_image[:, :] = resized[i, j]
        dim = (width, height)
        cart_image = cv2.resize(cart_image.astype(float), dim, interpolation=cv2.INTER_LINEAR)
        return cart_image

    ##### Algorithm on paper chun
    def car_to_pol_chun(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2
        polar_image = np.zeros_like(image)
        # Create grids for radius (r) and angle (θ)
        u = np.arange(height)
        ru = u / height
        v = np.arange(width)
        thetav = 2 * np.pi * v / width
        t, r = np.meshgrid(thetav, ru, indexing='xy')
        x = (r * width * np.cos(t) / 2).astype(int)
        y = (r * height * np.sin(t) / 2).astype(int)
        p = -x + center_x + 1
        q = y + center_y
        p = np.clip(p, 0, width - 1)
        q = np.clip(q, 0, height - 1)
        polar_image[:, :] = image[q, p]
        return polar_image

    def pol_to_car_chun(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2
        car_image = np.zeros_like(image)
        p = (np.arange(width) - center_x + 1) * 2 / width  # to make perfect square using +1
        q = (center_y - np.arange(height)) * 2 / height
        p, q = np.meshgrid(p, q, indexing="xy")
        ru = np.sqrt(p ** 2 + q ** 2)
        ru[ru > 1] = 0
        thetav = np.arctan2(q, p, where=True)
        # thetav[thetav < 0] = thetav[thetav < 0] + 2 * np.pi
        ru_index = (ru * height / 2).astype(int)
        theta_index = ((thetav + np.pi) * width / (2 * np.pi)).astype(int)
        i = np.clip(ru_index, 0, width - 1)
        j = np.clip(theta_index, 0, height - 1)
        # Map Cartesian image to polar image using advanced indexing
        car_image[:, :] = image[i, j]
        return car_image


###### scipy package

    def topolar(self, img, order=5):
        height, width = img.shape
        center_x, center_y = width // 2, height // 2
        # Calculate maximum radius for polar coordinates
        max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
        print("Radius : ", max_radius)
        # max_radius = 0.5*np.linalg.norm( img.shape )
        # print ("Radius : ", max_radius)
        def transform(coords):
            theta = 2.0 * np.pi * coords[1] / width
            radius = max_radius * coords[0] / height
            i = 0.5 * img.shape[0] - radius * np.sin(theta)
            j = radius * np.cos(theta) + 0.5 * img.shape[1]
            return i,j

        polar = geometric_transform(img, transform, order=order, mode='nearest', prefilter=True)
        return polar

    def tocart(self, img, order=5):
        height, width = img.shape
        center_x, center_y = width // 2, height // 2
        # Calculate maximum radius for polar coordinates
        max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
        print("Radius : ", max_radius)

        def transform(coords):
            xindex, yindex = coords
            #x0, y0 = (255, 255)
            x = xindex - center_x
            y = yindex - center_y
            r = np.sqrt(x ** 2.0 + y ** 2.0) * (img.shape[1] / max_radius)
            theta = np.arctan2(y, x, where=True)
            theta_index = (theta + np.pi) * img.shape[0] / (2 * np.pi)
            return (r, theta_index)

        cart = geometric_transform(img, transform, order=order, mode='nearest', prefilter=True)
        return cart

#### Linear opencv
    def cartesian_to_linear_polar(self,image):
      height, width = image.shape
      center_x, center_y = width // 2, height // 2
      # Calculate maximum radius for polar coordinates
      # max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
      min_dim = min(height, width)
      max_radius = min_dim / 2

      flags = cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS + cv2.WARP_POLAR_LINEAR
      linear_polar_image = cv2.warpPolar(image, (height, width), (center_x, center_y), max_radius, flags)
      return linear_polar_image

    def linear_polar_cartesian_image(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2
        # Calculate maximum radius for polar coordinates
        # max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
        min_dim = min(height, width)
        max_radius = min_dim / 2
        flags = cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS + cv2.WARP_POLAR_LINEAR + cv2.WARP_INVERSE_MAP
        inear_polar_image = cv2.warpPolar(image, (height, width), (center_x, center_y), max_radius, flags)
        return inear_polar_image


### log linear opencv
    def cartesian_to_linear_logpolar(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2

        # Calculate maximum radius for polar coordinates
        # max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
        min_dim = min(height, width)
        max_radius = min_dim / 2
        flags = cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS + cv2.WARP_POLAR_LOG
        linear_polar_image = cv2.warpPolar(image, (height, width), (center_x, center_y), max_radius, flags)
        return linear_polar_image

    def linear_log_polar_cartesian_image(self,image):
        height, width = image.shape
        center_x, center_y = width // 2, height // 2

        # Calculate maximum radius for polar coordinates
        # max_radius = int(np.sqrt(center_x ** 2 + center_y ** 2))
        min_dim = min(height, width)
        max_radius =  min_dim / 2
        flags = cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS + cv2.WARP_POLAR_LOG + cv2.WARP_INVERSE_MAP
        inear_polar_image = cv2.warpPolar(image, (height, width), (center_x, center_y), max_radius, flags)
        return inear_polar_image


    def display_image(self,title,oimage,pimage,rimage):
        # Visualize the original and polar images
        plt.figure(title,figsize=(10, 5))
        plt.subplot(1, 3, 1)
        plt.title("Original Image")
        plt.imshow(oimage, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Polar Coordinate Image")
        plt.imshow(pimage, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("inverse Polar Coordinate Image")
        plt.imshow(rimage, cmap="gray")
        plt.axis("off")

        plt.tight_layout()
        plt.show()


if __name__ == "__main__":
    # Load the image
    image = cv2.imread("lena/lena.jpg", cv2.IMREAD_GRAYSCALE)
    #image = cv2.resize(image, (128, 128))
    pit = PolarImageTransformation()
    # polar_image = pit.car_to_pol_zhang(image,1)
    # polar_image = pit.car_to_pol_chun(image)
    # # Convert to polar coordinates
    # developed algorithm
    # polar_image = pit.cartesian_to_polar_image(image)
    # inverse_polar_image = pit.polar_to_cartesian_image(polar_image)
    # title = "Manual"
    # msre = np.sum(np.square(image - inverse_polar_image)) / np.sum(np.square(image))
    # print("Reconstruction error : ", msre)

    # Based on paper modified
    # polar_image = pit.car_to_pol(image)
    # inverse_polar_image = pit.pol_to_car(polar_image)
    # msre = np.sum(np.square(image - inverse_polar_image)) / np.sum(np.square(image))
    # print("Reconstruction error : ", msre)
    # title = "Manual"

    # Based on scipy package
    # polar_image = pit.topolar(image)
    # inverse_polar_image = pit.tocart(polar_image)
    # title = "Scipy Package"

    # Based on OpenCv package
    polar_image = pit.cartesian_to_linear_polar(image)
    inverse_polar_image = pit.linear_polar_cartesian_image(polar_image)
    msre = np.sum(np.square(image - inverse_polar_image)) / np.sum(np.square(image))
    print("Reconstruction error : ", msre)
    title = "OpenCv Linear"

    # Based on OpenCv package
    # polar_image = pit.cartesian_to_linear_logpolar(image)
    # inverse_polar_image = pit.linear_log_polar_cartesian_image(polar_image)
    # title = "OpenCv log Linear"

    # inverse_polar_image = pit.pol_to_car_zhang(polar_image,1)
    # inverse_polar_image = pit.pol_to_car_chun(polar_image)
    msre = np.sum(np.square(image - inverse_polar_image)) / np.sum(np.square(image))
    print("Reconstruction error : ", msre)
    # title = "Chun"



    pit.display_image(title,image,polar_image, inverse_polar_image)





AttributeError: 'NoneType' object has no attribute 'shape'